### Imports

In [1]:
from mpl_toolkits import mplot3d
from sklearn.metrics import silhouette_score
from sklearn.model_selection import RandomizedSearchCV
from tqdm import tqdm
from utils.utils import *

import hdbscan
import matplotlib.pyplot as plt
import networkx as nx
import numpy as np
import os
import pandas as pd
import sys

%load_ext autotime

/home/ealvarez/miniconda3/envs/Conda_dgl_3.10/lib/python3.10/site-packages/umap/distances.py:1063: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  @numba.jit()
/home/ealvarez/miniconda3/envs/Conda_dgl_3.10/lib/python3.10/site-packages/umap/distances.py:1071: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  @numba.jit()
/home/ealvarez/miniconda3/envs/Conda_dgl_3.10/li

time: 175 µs (started: 2023-05-31 12:29:13 -05:00)


### Parameters

In [2]:
import json
  
# Opening JSON file
file = open("parameters.json")
params = json.load(file)

dir = os.path.dirname(os.getcwd())
print(dir)

method = params["method"][params["method_idx"]]
print("Method:\t\t", method)

group = params["group"][params["group_idx"]]
print("Group:\t\t", group)

subgroups = [str(k + 1) for k in range(group[1])]
print("Subgroup:\t", subgroups)

dimensions = params["dimensions"]
dimension = dimensions[0]
print("Dimensions:\t", dimensions)

/home/ealvarez/Project/GNN_Filter
Method:		 dgi
Group:		 ['WT', 5]
Subgroup:	 ['1', '2', '3', '4', '5']
Dimensions:	 [3]
time: 1.41 ms (started: 2023-05-31 12:29:13 -05:00)


In [3]:
list_graphs = []

time: 609 µs (started: 2023-05-31 12:29:13 -05:00)


### Get common subgraphs

In [4]:
graphs = []
for item in tqdm(subgroups):
    weighted_edges = pd.read_csv("{}/output_preprocessing/edges/{}_edges_{}.csv".format(dir, group[0], item))
    G = nx.from_pandas_edgelist(weighted_edges, "source", "target", edge_attr="weight")
    graph_detail(G)
    graphs.append(G)

 20%|██        | 1/5 [00:03<00:12,  3.04s/it]

Num. nodes: 6234
Num. edges: 1243057



 40%|████      | 2/5 [00:05<00:07,  2.54s/it]

Num. nodes: 6231
Num. edges: 939038



 60%|██████    | 3/5 [00:10<00:07,  3.83s/it]

Num. nodes: 6233
Num. edges: 2252847



 80%|████████  | 4/5 [00:19<00:05,  5.90s/it]

Num. nodes: 6226
Num. edges: 3728175



100%|██████████| 5/5 [00:30<00:00,  6.00s/it]

Num. nodes: 6226
Num. edges: 4071713

time: 30 s (started: 2023-05-31 12:29:13 -05:00)


In [5]:
SG = get_subgraphs(graphs)
graph_detail(SG)

list_graphs.append(SG.copy())
nx.write_gexf(SG, "{}/output_greedy/graphs_filter/greedy_{}_graph-filter.gexf".format(dir, group[0]))

100%|██████████| 4/4 [00:38<00:00,  9.66s/it]


Num. nodes: 4202
Num. edges: 135076

time: 49.6 s (started: 2023-05-31 12:29:43 -05:00)


In [6]:
df_edges_subgraph = pd.DataFrame(SG.edges())
df_edges_subgraph.columns = ["source", "target"]
df_edges_subgraph

,source,target
0,243.0404,267.1048
1,243.0404,401.2187
2,243.0404,661.3471
3,243.0404,267.0965
4,243.0404,416.1414
...,...,...
135071,579.4245,579.4980
135072,663.3295,663.3449
135073,226.0573,226.0718
135074,412.8122,412.8271


time: 98.8 ms (started: 2023-05-31 12:30:33 -05:00)


### Calculate ANOVA (nodes)

In [7]:
""" df_nodes_anova = get_nodes_anova(SG, dir, group)
df_nodes_anova

# Filter by ANOVA (p > 0.001)
df_nodes_anova_filter = df_nodes_anova[df_nodes_anova["p-value"] > 0.001]
df_nodes_anova_filter
nodes = list(df_nodes_anova_filter.index)
len(nodes) """

' df_nodes_anova = get_nodes_anova(SG, dir, group)\ndf_nodes_anova\n\n# Filter by ANOVA (p > 0.001)\ndf_nodes_anova_filter = df_nodes_anova[df_nodes_anova["p-value"] > 0.001]\ndf_nodes_anova_filter\nnodes = list(df_nodes_anova_filter.index)\nlen(nodes) '

time: 1.94 ms (started: 2023-05-31 12:30:33 -05:00)


### Calculate STD (edges)

In [8]:
""" df_edges_std = get_edges_std(SG, dir, group, subgroups)
df_edges_std

# Filter by std (std <= 0.3)
df_edges_std_filter = df_edges_std[df_edges_std["std"] <= 0.3]
df_edges_std_filter """

' df_edges_std = get_edges_std(SG, dir, group, subgroups)\ndf_edges_std\n\n# Filter by std (std <= 0.3)\ndf_edges_std_filter = df_edges_std[df_edges_std["std"] <= 0.3]\ndf_edges_std_filter '

time: 1.69 ms (started: 2023-05-31 12:30:33 -05:00)


---

### STD-ANOVA

In [9]:
# STD
df_edges_std = get_edges_std(SG, dir, group, subgroups, ddof=0)
df_edges_std

# Filter by std (std < 0.3)
df_edges_std_filter = df_edges_std[df_edges_std["std"] < 0.3]
df_edges_std_filter

 80%|████████  | 4/5 [00:11<00:03,  3.62s/it]

In [ ]:
G = nx.from_pandas_edgelist(df_edges_std_filter, "source", "target", edge_attr=["weight"])
graph_detail(G)

list_graphs.append(G.copy())
nx.write_gexf(G, "{}/output_greedy/graphs_filter/greedy_{}_graph-filter-std.gexf".format(dir, group[0]))

Num. nodes: 5677
Num. edges: 732148

time: 16.1 s (started: 2023-05-29 23:56:47 -05:00)


In [ ]:
df_edges_filter = nx.to_pandas_edgelist(G)
df_edges_filter.to_csv("{}/output_greedy/edges_filter/greedy_{}_edge-filter-std.csv".format(dir, group[0]), index=False)
df_edges_filter

,source,target,weight
0,59.0137,59.0291,0.974889
1,59.0137,59.0370,0.740804
2,59.0137,60.0171,0.801043
3,59.0137,61.9884,0.684453
4,59.0137,71.0139,0.681361
...,...,...,...
732143,978.3602,978.3779,0.682080
732144,978.4185,978.6600,0.754314
732145,979.8201,979.8546,0.664992
732146,979.8546,979.9144,0.614228


time: 8.88 s (started: 2023-05-29 23:57:03 -05:00)


In [ ]:
# ANOVA
df_nodes_anova = get_nodes_anova(G, dir, group)
df_nodes_anova

# Filter by ANOVA (p > 0.001)
df_nodes_anova_filter = df_nodes_anova[df_nodes_anova["p-value"] > 0.001]
df_nodes_anova_filter
nodes = list(df_nodes_anova_filter.index)

time: 10.2 s (started: 2023-05-29 23:57:12 -05:00)


In [ ]:
H = G.subgraph(nodes)
graph_detail(H)
print(len(list(n for n in H.nodes() if H.degree(n) == 0)))

list_graphs.append(H.copy())
nx.write_gexf(H, "{}/output_greedy/graphs_filter/greedy_{}_graph-filter-std-anova.gexf".format(dir, group[0]))

df_edges_filter = nx.to_pandas_edgelist(H)

df_edges_filter.to_csv("{}/output_greedy/edges_filter/greedy_{}_edge-filter-std-anova.csv".format(dir, group[0]), index=False)
df_edges_filter

Num. nodes: 4691
Num. edges: 473180

40


,source,target,weight
0,59.0137,59.0291,0.974889
1,59.0137,59.0370,0.740804
2,59.0137,60.0171,0.801043
3,59.0137,61.9884,0.684453
4,59.0137,71.0139,0.681361
...,...,...,...
473175,978.3602,978.3779,0.682080
473176,978.4185,978.6600,0.754314
473177,979.8201,979.8546,0.664992
473178,979.8546,979.9144,0.614228


time: 24.7 s (started: 2023-05-29 23:57:23 -05:00)


### ANOVA-STD

In [ ]:
# ANOVA
df_nodes_anova = get_nodes_anova(SG, dir, group)
df_nodes_anova

# Filter by ANOVA (p > 0.001)
df_nodes_anova_filter = df_nodes_anova[df_nodes_anova["p-value"] > 0.001]
df_nodes_anova_filter
nodes = list(df_nodes_anova_filter.index)

time: 10.1 s (started: 2023-05-29 23:57:47 -05:00)


In [ ]:
H = SG.subgraph(nodes)

time: 3.6 ms (started: 2023-05-29 23:57:58 -05:00)


In [ ]:
df_edges_filter = nx.to_pandas_edgelist(H)
df_edges_filter.to_csv("{}/output_greedy/edges_filter/greedy_{}_edge-filter-anova.csv".format(dir, group[0]), index=False)
df_edges_filter

,source,target
0,520.9177,378.7450
1,520.9177,397.2518
2,520.9177,147.0655
3,520.9177,241.9185
4,520.9177,333.2457
...,...,...
524839,455.0738,455.1677
524840,711.1266,711.1610
524841,687.0800,687.0919
524842,760.3625,760.3770


time: 8.52 s (started: 2023-05-29 23:57:58 -05:00)


In [ ]:
H = nx.from_pandas_edgelist(df_edges_filter, "source", "target")
graph_detail(H)

list_graphs.append(H.copy())
nx.write_gexf(H, "{}/output_greedy/graphs_filter/greedy_{}_graph-filter-anova.gexf".format(dir, group[0]))

Num. nodes: 4674
Num. edges: 524844

time: 9.99 s (started: 2023-05-29 23:58:06 -05:00)


In [ ]:
# STD
df_edges_std = get_edges_std(H, dir, group, subgroups, ddof=0)
df_edges_std

# Filter by std (std < 0.3)
df_edges_std_filter = df_edges_std[df_edges_std["std"] < 0.3]
df_edges_std_filter

100%|██████████| 2/2 [00:08<00:00,  4.10s/it]


,source,target,weight,std
51879,59.0137,59.0291,0.974889,0.013730
165185,59.0137,59.0370,0.740804,0.087645
164684,59.0137,60.0171,0.801043,0.096319
36831,59.0137,61.9884,0.684453,0.095850
120370,59.0137,71.0139,0.681361,0.020293
...,...,...,...,...
181890,984.5048,984.7590,0.673136,0.046539
182021,984.7590,986.7752,0.651981,0.133005
182024,984.7590,987.7787,0.627558,0.088982
509966,985.7644,986.7752,0.581169,0.054479


time: 9.53 s (started: 2023-05-29 23:58:16 -05:00)


In [ ]:
G = nx.from_pandas_edgelist(df_edges_std_filter, "source", "target", edge_attr=["weight"])
graph_detail(G)

list_graphs.append(G.copy())
nx.write_gexf(G, "{}/output_greedy/graphs_filter/greedy_{}_graph-filter-anova-std.gexf".format(dir, group[0]))

Num. nodes: 4651
Num. edges: 473180

time: 9.55 s (started: 2023-05-29 23:58:26 -05:00)


In [ ]:
df_edges_filter = nx.to_pandas_edgelist(G)
df_edges_filter.to_csv("{}/output_greedy/edges_filter/greedy_{}_edge-filter-anova-std.csv".format(dir, group[0]), index=False)
df_edges_filter

,source,target,weight
0,59.0137,59.0291,0.974889
1,59.0137,59.0370,0.740804
2,59.0137,60.0171,0.801043
3,59.0137,61.9884,0.684453
4,59.0137,71.0139,0.681361
...,...,...,...
473175,978.3602,978.3779,0.682080
473176,978.4185,978.6600,0.754314
473177,979.8201,979.8546,0.664992
473178,979.8546,979.9144,0.614228


time: 7.53 s (started: 2023-05-29 23:58:36 -05:00)


### Results

In [ ]:
for graph in list_graphs:
    graph_detail(graph)

Num. nodes: 5710
Num. edges: 842720

Num. nodes: 5677
Num. edges: 732148

Num. nodes: 4691
Num. edges: 473180

Num. nodes: 4674
Num. edges: 524844

Num. nodes: 4651
Num. edges: 473180

time: 24.9 ms (started: 2023-05-29 23:58:43 -05:00)
